In [3]:
pip install schrutepy transformers plotly nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.2/769.2 kB 41.7 MB/s eta 0:00:00
  Created wheel for schrutepy: filename=schrutepy-0.1.3-py3-none-any.whl size=3472 sha256=4a796e84fc83d6d6f5864cc6e7cfdee19cb850c40ff224fe7fffc8731a7987d1
  Stored in directory: /root/.cache/pip/wheels/2b/d8/94/ec3e1ca6b4df2efdd059b5a1ace8e12b223ac816edd9634470
Successfully built schrutepy
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 5.0.2

In [4]:
from schrutepy import schrutepy
import plotly.express as px
import pandas as pd
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

import nltk
from nltk.corpus import stopwords
from nltk.corpus import opinion_lexicon
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Carga de guiones
df = schrutepy.load_schrute()
df.set_index('index', inplace=True)
df['text'] = df['text'].astype(str)
df.head(5)

,season,episode,episode_name,director,writer,character,text,text_w_direction
index,,,,,,,,
1,1,1,Pilot,Ken Kwapis,Ricky Gervais;Stephen Merchant;Greg Daniels,Michael,All right Jim. Your quarterlies look very good...,All right Jim. Your quarterlies look very good...
2,1,1,Pilot,Ken Kwapis,Ricky Gervais;Stephen Merchant;Greg Daniels,Jim,"Oh, I told you. I couldn't close it. So...","Oh, I told you. I couldn't close it. So..."
3,1,1,Pilot,Ken Kwapis,Ricky Gervais;Stephen Merchant;Greg Daniels,Michael,So you've come to the master for guidance? Is ...,So you've come to the master for guidance? Is ...
4,1,1,Pilot,Ken Kwapis,Ricky Gervais;Stephen Merchant;Greg Daniels,Jim,"Actually, you called me in here, but yeah.","Actually, you called me in here, but yeah."
5,1,1,Pilot,Ken Kwapis,Ricky Gervais;Stephen Merchant;Greg Daniels,Michael,"All right. Well, let me show you how it's done.","All right. Well, let me show you how it's done."


In [ ]:
df.describe()


,season,episode
count,55130.00000,55130.000000
mean,5.53869,12.547978
std,2.34264,7.295413
min,1.00000,1.000000
25%,3.00000,6.000000
50%,6.00000,12.000000
75%,7.00000,19.000000
max,9.00000,28.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55130 entries, 1 to 55130
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   season            55130 non-null  int64 
 1   episode           55130 non-null  int64 
 2   episode_name      55130 non-null  object
 3   director          55130 non-null  object
 4   writer            55130 non-null  object
 5   character         55130 non-null  object
 6   text              55130 non-null  object
 7   text_w_direction  55125 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.8+ MB


In [ ]:
# Contar cantidad de lineas dichas por cada personaje
character_lines = df['character'].value_counts().reset_index()
character_lines.columns = ['character', 'line_count']

# Top 10 personajes
top_10_characters = character_lines.head(10)

# Grafico de barras
fig = px.bar(top_10_characters,
             x='character',
             y='line_count',
             title='Quien habla mas?',
             labels={'character': 'Personaje', 'line_count': 'Cantidad de lineas'})
fig.update_traces(marker_color='#4169E1')
fig.show()

In [ ]:
# Filtrando lineas de Dwhigt
dwight_df = df[df['character'] == 'Dwight']
all_text = ' '.join(dwight_df['text'].astype(str).tolist())

In [ ]:
# Cargando stop words
stop_words = set(stopwords.words('english'))

# Tokenize the text while preserving contractions
tokens = re.findall(r"\b\w[\w']*\b", all_text)

# Exclude stop words and preserve
filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

In [ ]:
# Create unigrams, bigrams, and trigrams from the filtered tokens
unigrams = filtered_tokens
bigrams = [' '.join(bigram) for bigram in zip(filtered_tokens, filtered_tokens[1:])]
trigrams = [' '.join(trigram) for trigram in zip(filtered_tokens, filtered_tokens[1:], filtered_tokens[2:])]

# Function to filter out n-grams with repeated words
def filter_repeated_ngrams(ngrams):
    pattern = re.compile(r'\b(\w+)\b(?: \1\b)+')
    return [ngram for ngram in ngrams if not pattern.search(ngram)]

# Filtrado de repetidos
bigrams = filter_repeated_ngrams(bigrams)
trigrams = filter_repeated_ngrams(trigrams)

# Conteo
unigram_counts = Counter(unigrams)
bigram_counts = Counter(bigrams)
trigram_counts = Counter(trigrams)

In [ ]:
# Obtener los 10 unigramas, bigramas y trigramas más comunes
most_common_unigrams = unigram_counts.most_common(20)
most_common_bigrams = bigram_counts.most_common(20)
most_common_trigrams = trigram_counts.most_common(20)

# convertir los unigramas, bigramas y trigramas más comunes a DataFrames
df_unigrams = pd.DataFrame(most_common_unigrams, columns=['Unigram', 'Count'])
df_bigrams = pd.DataFrame(most_common_bigrams, columns=['Bigram', 'Count'])
df_trigrams = pd.DataFrame(most_common_trigrams, columns=['Trigram', 'Count'])

# Graficar unigramas
fig_unigrams = px.bar(df_unigrams, x='Unigram', y='Count', title=' Dwight Top 10 Unigrams')
fig_unigrams.update_traces(marker_color='darkgreen')
fig_unigrams.show()

# Graficar bigramas
fig_bigrams = px.bar(df_bigrams, x='Bigram', y='Count', title='Dwight Top 10 Bigrams')
fig_bigrams.update_traces(marker_color='darkgreen')
fig_bigrams.show()

# Graficar trigramas
fig_trigrams = px.bar(df_trigrams, x='Trigram', y='Count', title='Dwight Top 10 Trigrams')
fig_trigrams.update_traces(marker_color='darkgreen')
fig_trigrams.show()

In [ ]:
sentiment_analyzer = pipeline('sentiment-analysis')

df['transformer_sentiment'] = df['text'].apply(lambda x: sentiment_analyzer(x)[0])
df['transformer_sentiment_label'] = df['transformer_sentiment'].apply(lambda x: x['label'])
df['transformer_sentiment_score'] = df['transformer_sentiment'].apply(lambda x: x['score'])

df[['character', 'text', 'transformer_sentiment']].head(5)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



,character,text,transformer_sentiment
index,,,
1,Michael,All right Jim. Your quarterlies look very good...,"{'label': 'POSITIVE', 'score': 0.9996935129165..."
2,Jim,"Oh, I told you. I couldn't close it. So...","{'label': 'NEGATIVE', 'score': 0.9978529214859..."
3,Michael,So you've come to the master for guidance? Is ...,"{'label': 'NEGATIVE', 'score': 0.9943307638168..."
4,Jim,"Actually, you called me in here, but yeah.","{'label': 'POSITIVE', 'score': 0.9541692137718..."
5,Michael,"All right. Well, let me show you how it's done.","{'label': 'POSITIVE', 'score': 0.9993804693222..."


In [ ]:
# Filtrar top 10 personajes
character_sentence_counts = df['character'].value_counts().head(20)
top_characters = character_sentence_counts.index.tolist()
df_top_characters = df[df['character'].isin(top_characters)]

# Calcular los porcentajes de sentimiento
sentiment_counts = df_top_characters.groupby(['character', 'transformer_sentiment_label']).size().unstack(fill_value=0)

# Calcular los porcentajes de sentimiento para cada personaje
sentiment_counts['Total'] = sentiment_counts.sum(axis=1)

# Calcular la proporción de sentimientos positivos y negativos
sentiment_counts['Positive'] = 100 * sentiment_counts['POSITIVE'] / sentiment_counts['Total']
sentiment_counts['Negative'] = 100 * sentiment_counts['NEGATIVE'] / sentiment_counts['Total']

In [ ]:
sentiment_counts = sentiment_counts.reset_index()
sentiment_counts = sentiment_counts.sort_values(by='Positive', ascending=False)

# Gráficar de barras horizontales apiladas
color_map = {'Positive': '#228B22', 'Negative': '#FF6347'}
fig = px.bar(sentiment_counts,
             x=['Positive', 'Negative'],
             y='character',
             orientation='h',
             title='Porcentaje de palabras con sentimientos positivos y negativos por carácter (10 caracteres principales)',
             labels={'value': 'Porcentaje (%)', 'character': 'Personaje'},
             height=600,
             width=1200,
             color_discrete_map=color_map,
)
fig.update_traces(hovertemplate='%{x:.2f}%')
fig.update_layout(barmode='stack', xaxis={'categoryorder': 'total descending'}, showlegend=True)
fig.show()

In [ ]:
model_name = "monologg/bert-base-cased-goemotions-original"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
emotion_classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True)

tokenizer_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning:

`return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.



In [ ]:
# Analisis de emociones
def classify_emotions(text):
    emotions = emotion_classifier(text)[0]
    return {emotion['label']: emotion['score'] for emotion in emotions}

df['emotions'] = df['text'].apply(classify_emotions)
emotion_labels = emotion_classifier(df['text'].iloc[0])[0]

for emotion in emotion_labels:
    df[emotion['label']] = df['emotions'].apply(lambda x: x.get(emotion['label'], 0))

In [ ]:
# Contar frases por personaje
character_sentence_counts = df['character'].value_counts().head(10)

# Filtrar los 10 primeros personajes
top_characters = character_sentence_counts.index.tolist()
df_top_characters = df[df['character'].isin(top_characters)]

# Calcular las puntuaciones totales de cada emoción por personaje
emotion_totals = df_top_characters.groupby('character')[emotion_labels[0]['label']].sum().reset_index()
for emotion in emotion_labels[1:]:
    emotion_totals = emotion_totals.merge(df_top_characters.groupby('character')[emotion['label']].sum().reset_index(), on='character')

#Convertir totales en porcentajes
emotion_totals = emotion_totals.drop(columns=['neutral'])
emotion_totals.set_index('character', inplace=True)
emotion_totals = emotion_totals.div(emotion_totals.sum(axis=1), axis=0).multiply(100)
emotion_totals = emotion_totals.reset_index()

In [ ]:
melted_emotion_totals = emotion_totals.melt(id_vars=['character'], var_name='emotion', value_name='share')

# graficar Mapa de calor
fig = px.density_heatmap(melted_emotion_totals,
                         x='emotion',
                         y='character',
                         z='share',
                         title='Heatmap of Emotions by Character (Top 10 Characters)',
                         labels={'share': 'Share (%)', 'character': 'Character', 'emotion': 'Emotion'},
                         height=600,
                         width=1000,
                         color_continuous_scale='dense'
                        )
fig.show()

In [ ]:
Dwight_emotion_totals = emotion_totals[emotion_totals['character'] == 'Dwight'].reset_index(drop = True)
Dwight_emotion_totals = Dwight_emotion_totals.T
Dwight_emotion_totals.columns = Dwight_emotion_totals.iloc[0]
Dwight_emotion_totals = Dwight_emotion_totals.drop('character')
Dwight_emotions_df = Dwight_emotion_totals.reset_index()
Dwight_emotions_df.columns = ['emotion', 'share']

# Ordenar de mayor a menor
Dwight_emotions_df = Dwight_emotions_df.sort_values(by='share', ascending=True)

# Gráfico de barras
fig = px.bar(Dwight_emotions_df,
             x='share',
             y='emotion',
             orientation='h',
             title='Distribución de emociones para Dwight',
             labels={'share': 'Share (%)', 'emotion': 'Emotion'},
             text='share',
             height=600,
             width=1000)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis={'categoryorder': 'total descending'}, showlegend=False)
fig.show()

In [ ]:
# Calcular el sentimiento de cada línea
dwight_df['sentiment'] = dwight_df['text'].apply(lambda x: sentiment_analyzer(x)[0])
dwight_df['label'] = dwight_df['sentiment'].apply(lambda x: x['label'])  # Etiquetas de sentimiento (POSITIVE/NEGATIVE)
dwight_df['score'] = dwight_df['sentiment'].apply(lambda x: x['score'])  # Confianza en el sentimiento

# Agrupar los sentimientos de Dwight por temporada y contar positivos y negativos
sentiment_counts = dwight_df.groupby(['season', 'label']).size().unstack(fill_value=0)

# Calcular el total de líneas por temporada
sentiment_counts['Total'] = sentiment_counts.sum(axis=1)

# Calcular el porcentaje de líneas positivas y negativas
sentiment_counts['Positive'] = 100 * sentiment_counts['POSITIVE'] / sentiment_counts['Total']
sentiment_counts['Negative'] = 100 * sentiment_counts['NEGATIVE'] / sentiment_counts['Total']


<ipython-input-21-67486aa31d0f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-67486aa31d0f>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-67486aa31d0f>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Crear gráfico de barras apiladas en Plotly
color_map = {'Positive': '#228B22', 'Negative': '#FF6347'}
fig = px.bar(
    sentiment_counts.reset_index(),  # Restablecemos el índice para convertir la columna 'season' en parte del DataFrame
    x='season',  # Eje X es la temporada
    y=['Positive', 'Negative'],  # Las dos columnas que queremos apilar (Positive y Negative)
    title='Evolución del Sentimiento de Dwight por Temporada',
    labels={'value': 'Porcentaje', 'season': 'Temporada'},  # Etiquetas personalizadas
    height=400,
    width=800,
    color_discrete_map={'Positive': 'green', 'Negative': 'red'}
)

# Configurar apilamiento
fig.update_layout(barmode='stack')

# Mostrar gráfico
fig.show()


In [ ]:
# Calcular el sentimiento de cada línea para Dwight
dwight_df['sentiment'] = dwight_df['text'].apply(lambda x: sentiment_analyzer(x)[0])
dwight_df['label'] = dwight_df['sentiment'].apply(lambda x: x['label'])  # Etiquetas de sentimiento (POSITIVE/NEGATIVE)
dwight_df['score'] = dwight_df['sentiment'].apply(lambda x: x['score'])  # Confianza en el sentimiento

others_df = df[df['character'] != 'Dwight']

# Calcular el sentimiento de cada línea para los otros
others_df['sentiment'] = others_df['text'].apply(lambda x: sentiment_analyzer(x)[0])
others_df['label'] = others_df['sentiment'].apply(lambda x: x['label'])  # Etiquetas de sentimiento (POSITIVE/NEGATIVE)
others_df['score'] = others_df['sentiment'].apply(lambda x: x['score'])  # Confianza en el sentimiento

# Calcular porcentaje de comentarios positivos para Dwight
dwight_sentiment_counts = dwight_df['label'].value_counts(normalize=True) * 100
dwight_positive = dwight_sentiment_counts.get('POSITIVE', 0)

# Calcular el promedio de comentarios positivos para otros personajes
others_sentiment_counts = others_df['label'].value_counts(normalize=True) * 100
others_positive = others_sentiment_counts.get('POSITIVE', 0)

# Crear un DataFrame para comparar
comparison_df = pd.DataFrame({
    'Dwight': [dwight_positive],
    'Otros Personajes': [others_positive]
})

<ipython-input-23-3c06853a1e09>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-23-3c06853a1e09>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-23-3c06853a1e09>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-23-3c06853

In [ ]:
# Crear un gráfico de barras en Plotly
fig = px.bar(
    comparison_df.T.reset_index(),  # Transponemos y restablecemos el índice
    x='index',  # El eje X es el índice transpuesto ('Dwight' y 'Otros Personajes')
    y=0,  # La columna 0 contiene los valores del porcentaje
    title='Comparación de Comentarios Positivos: Dwight vs Otros Personajes',
    labels={'0': 'Porcentaje', 'index': 'Personajes'},  # Etiquetas personalizadas
    height=400,
    width=600,
    color='index',  # Colorea según el valor de la categoría (Dwight y Otros Personajes)
    color_discrete_map={'Dwight': 'blue', 'Otros Personajes': 'purple'}  # Asignar colores personalizados
)

# Mostrar el gráfico
fig.show()


In [ ]:
# Filtrar líneas donde Dwight se disculpa o corrige
df_apologies = dwight_df[dwight_df['text'].str.contains("sorry|apologize|mistake", case=False, na=False)]

# Calcular el sentimiento de esas líneas
df_apologies['sentiment'] = df_apologies['text'].apply(lambda x: sentiment_analyzer(x)[0])

# Extraer solo las etiquetas de sentimiento
df_apologies['sentiment_label'] = df_apologies['sentiment'].apply(lambda x: x['label'])

# Analizar emociones durante disculpas
df_apologies['emotions'] = df_apologies['text'].apply(classify_emotions)

# Convertir las emociones a DataFrame
emotions_df = pd.DataFrame(df_apologies['emotions'].tolist())

# Calcular el promedio de cada emoción
emotions_mean_df = emotions_df.mean().reset_index()
emotions_mean_df.columns = ['emotion', 'share']  # Renombrar columnas

# Ordenar de menor a mayor para una mejor visualización
emotions_mean_df = emotions_mean_df.sort_values(by='share', ascending=True)

# Crear gráfico de barras para mostrar las emociones durante las disculpas
fig = px.bar(
    emotions_mean_df,
    x='share',
    y='emotion',
    orientation='h',
    title='Distribución de emociones durante las disculpas de Dwight',
    labels={'share': 'Porcentaje (%)', 'emotion': 'Emociones'},
    text='share',
    height=600,
    width=1000
)

# Configurar el formato del texto y las etiquetas
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis={'categoryorder': 'total descending'}, showlegend=False)

# Mostrar el gráfico
fig.show()


<ipython-input-26-e2b85eb79b25>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-e2b85eb79b25>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-e2b85eb79b25>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

